# Association Rules

Associatio rules are frequently used for Market Basket Analysis (MBA) by retailers to understand the purchase behavior of their customers. This information can be then used for many different purposes such as cross-selling and up-selling of product, sales promotions, loyalty programs, store design, discount plans and many others

**Evaluation of item set**: Once you have found the frequent itemsets of a dataset, you need to choose a subset of them as your recommendations. 

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 217.8MB 63kB/s 
     |████████████████████████████████| 204kB 48.1MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218257927 sha256=81fdccb64a81cf9ebdfc63600a4121daa7b3e36025c35a3a75a44a47f3c4349a
  Stored in directory: /root/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark
openjdk-8-jdk-headless is already the newest version (8u242-b08-0ubuntu3~18.04).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd

# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

## A-prior Algorithm

In [ ]:
import numpy as np

In [ ]:
browsing = sc.textFile("browsing.txt")
browsing.take(2)

['FRO11987 ELE17451 ELE89019 SNA90258 GRO99222 ',
 'GRO99222 GRO12298 FRO12685 ELE91550 SNA11465 ELE26917 ELE52966 FRO90334 SNA30755 ELE17451 FRO84225 SNA80192 ']

In [ ]:
# convert string into a list of items for each basket
items = browsing.map(lambda x: x.split(" "))
items.take(2)

[['FRO11987', 'ELE17451', 'ELE89019', 'SNA90258', 'GRO99222', ''],
 ['GRO99222',
  'GRO12298',
  'FRO12685',
  'ELE91550',
  'SNA11465',
  'ELE26917',
  'ELE52966',
  'FRO90334',
  'SNA30755',
  'ELE17451',
  'FRO84225',
  'SNA80192',
  '']]

In [ ]:
# remove duplicate items (if any) in each busket
items_no_dupes = items.map(lambda x: list(set(x)))
items_no_dupes.take(2)

[['ELE17451', '', 'FRO11987', 'SNA90258', 'ELE89019', 'GRO99222'],
 ['ELE17451',
  'SNA11465',
  '',
  'FRO90334',
  'SNA80192',
  'SNA30755',
  'FRO12685',
  'GRO12298',
  'ELE91550',
  'FRO84225',
  'ELE26917',
  'ELE52966',
  'GRO99222']]

In [ ]:
# consolidate all items in all busktets for first step of A-priori
items_list = items_no_dupes.flatMap(lambda x:x)
items_list.take(5)

['ELE17451', '', 'FRO11987', 'SNA90258', 'ELE89019']

In [ ]:
# remove empty entries in the list
all_items = items_list.filter(lambda x: x!='')
all_items.take(10)

['ELE17451',
 'FRO11987',
 'SNA90258',
 'ELE89019',
 'GRO99222',
 'ELE17451',
 'SNA11465',
 'FRO90334',
 'SNA80192',
 'SNA30755']

In [ ]:
# construct key-value pair for each item in the list, with value=1
item_pairs = all_items.map(lambda x: (x,1))
item_pairs.take(10)

[('ELE17451', 1),
 ('FRO11987', 1),
 ('SNA90258', 1),
 ('ELE89019', 1),
 ('GRO99222', 1),
 ('ELE17451', 1),
 ('SNA11465', 1),
 ('FRO90334', 1),
 ('SNA80192', 1),
 ('SNA30755', 1)]

In [ ]:
# count occurance of each item in the list
item_counts = item_pairs.reduceByKey(lambda a,b:a+b)
item_counts.take(10)

[('FRO11987', 104),
 ('SNA90258', 550),
 ('FRO90334', 63),
 ('SNA80192', 258),
 ('ELE91550', 23),
 ('FRO84225', 74),
 ('ELE52966', 380),
 ('GRO73461', 3602),
 ('DAI22896', 1219),
 ('FRO86643', 235)]

In [ ]:
# filter out items with support>=100
frequent_items_counts = item_counts.filter(lambda x: x[1]>= 100)
frequent_items_counts.take(10)

[('FRO11987', 104),
 ('SNA90258', 550),
 ('SNA80192', 258),
 ('ELE52966', 380),
 ('GRO73461', 3602),
 ('DAI22896', 1219),
 ('FRO86643', 235),
 ('SNA69641', 599),
 ('GRO39357', 296),
 ('ELE11375', 214)]

In [ ]:
# sanity check: 647 frequent items
frequent_items_counts.count() == 647

True

In [ ]:
# get a list of frequent items
frequent_item_list = frequent_items_counts.map(lambda x: x[0]).collect()
frequent_item_list[:5]

['FRO11987', 'SNA90258', 'SNA80192', 'ELE52966', 'GRO73461']

In [ ]:
# remove non-frequent items in each basket
basket_of_frequent_items = items_no_dupes.map(lambda x: [i for i in x if i in frequent_item_list])
basket_of_frequent_items.take(3)

[['ELE17451', 'FRO11987', 'SNA90258', 'GRO99222'],
 ['ELE17451',
  'SNA11465',
  'SNA80192',
  'SNA30755',
  'GRO12298',
  'ELE26917',
  'ELE52966',
  'GRO99222'],
 ['GRO73461', 'DAI22896', 'ELE17451', 'FRO86643', 'SNA99873']]

In [ ]:
# sort frequent items in each basket 
# (ordered items will avoid duplicates in (X,Y) pairs)
basket_of_frequent_items_sorted = basket_of_frequent_items.map(lambda x: sorted(x))
basket_of_frequent_items_sorted.take(2)

[['ELE17451', 'FRO11987', 'GRO99222', 'SNA90258'],
 ['ELE17451',
  'ELE26917',
  'ELE52966',
  'GRO12298',
  'GRO99222',
  'SNA11465',
  'SNA30755',
  'SNA80192']]

In [ ]:
# create pairs of frequent items in each basket
# note that each basket is ordered
pair_freq_items_each_basket = basket_of_frequent_items_sorted.map(lambda x: [(i,j) for i in x for j in x if i<j])
pair_freq_items_each_basket.take(1)

[[('ELE17451', 'FRO11987'),
  ('ELE17451', 'GRO99222'),
  ('ELE17451', 'SNA90258'),
  ('FRO11987', 'GRO99222'),
  ('FRO11987', 'SNA90258'),
  ('GRO99222', 'SNA90258')]]

In [ ]:
# sanity check:
# the number of pairs of frequent items in each basket = n (basket size) choose 2
num_pairs_freq_items_per_basket = pair_freq_items_each_basket.map(lambda x: len(x)).collect()
basket_size_choose_2 = basket_of_frequent_items_sorted.map(lambda x: int(len(x)*(len(x)-1)/2)).collect()
# number of descripancies (want 0)
np.sum([basket_size_choose_2[i]!=num_pairs_freq_items_per_basket[i] for i in range(len(basket_size_choose_2))])

0

In [ ]:
# get a list of all pairs of frequent items 
list_pairs_freq_items = pair_freq_items_each_basket.flatMap(lambda x:x).collect()
list_pairs_freq_items[:5]

[('ELE17451', 'FRO11987'),
 ('ELE17451', 'GRO99222'),
 ('ELE17451', 'SNA90258'),
 ('FRO11987', 'GRO99222'),
 ('FRO11987', 'SNA90258')]

In [ ]:
# count pairs in the list
from collections import Counter
pair_of_frequent_items_count = dict(Counter(list_pairs_freq_items))
pair_of_frequent_items_count

In [ ]:
# extract pairs that have support >= 100
frequent_pairs = [[key, val] for key, val in pair_of_frequent_items_count.items() if val >= 100]
frequent_pairs[:5]

[[('ELE17451', 'GRO99222'), 148],
 [('ELE17451', 'SNA90258'), 113],
 [('GRO99222', 'SNA90258'), 156],
 [('ELE17451', 'ELE26917'), 314],
 [('ELE17451', 'SNA30755'), 111]]

In [ ]:
# construct pd dataframe [table1] of frequent pairs and counts
frequent_pairs_pd = pd.DataFrame(frequent_pairs).rename(columns={0:"freq_pairs", 1:"pair_count"})
nrow = frequent_pairs_pd.shape[0]
frequent_pairs_pd["freq_item_A"] = [frequent_pairs_pd.freq_pairs[i][0] for i in range(nrow)]
frequent_pairs_pd["freq_item_B"] = [frequent_pairs_pd.freq_pairs[i][1] for i in range(nrow)]
frequent_pairs_pd.head(3)

,freq_pairs,pair_count,freq_item_A,freq_item_B
0,"(ELE17451, GRO99222)",148,ELE17451,GRO99222
1,"(ELE17451, SNA90258)",113,ELE17451,SNA90258
2,"(GRO99222, SNA90258)",156,GRO99222,SNA90258


In [ ]:
# construct pd dataframe [table2] of frequent items and counts
frequent_item_pd = frequent_items_counts.toDF(("item","item_count")).toPandas()
frequent_item_pd.head(3)

,item,item_count
0,FRO11987,104
1,SNA90258,550
2,SNA80192,258


In [ ]:
# join [table1] and [table2] to calculate conf(A \rightarrow B)
confidence_df = pd.merge(frequent_pairs_pd, frequent_item_pd, 
                        how="left", left_on="freq_item_A", right_on = "item") \
                  .drop(columns=['item']) \
                  .rename(columns={"item_count":"item_count_A"}) \
                  .merge(frequent_item_pd, how="left", left_on="freq_item_B", right_on = "item") \
                  .drop(columns=['item', 'freq_pairs'])\
                  .rename(columns={"item_count":"item_count_B"}) 
confidence_df.head(3)

,pair_count,freq_item_A,freq_item_B,item_count_A,item_count_B
0,148,ELE17451,GRO99222,3875,906
1,113,ELE17451,SNA90258,3875,550
2,156,GRO99222,SNA90258,906,550


In [ ]:
confidence_reverse_df = confidence_df.iloc[:, [0,2,1,4,3]] \
                                     .rename(columns={"freq_item_A":"freq_item_B",
                                                      "freq_item_B":"freq_item_A",
                                                      "item_count_B":"item_count_A",
                                                      "item_count_A":"item_count_B"})
confidence_reverse_df.head(3)                                

,pair_count,freq_item_A,freq_item_B,item_count_A,item_count_B
0,148,GRO99222,ELE17451,906,3875
1,113,SNA90258,ELE17451,550,3875
2,156,SNA90258,GRO99222,550,906


In [ ]:
confidence_df = confidence_df.append(confidence_reverse_df)

In [ ]:
confidence_df["confidence"]=confidence_df.pair_count/confidence_df.item_count_A

In [ ]:
# sort table by confidence score (desc) and item A (asc)
confidence_df.sort_values(["confidence", "freq_item_A"], ascending = (False, True), inplace=True)

# 2d) outputs
print("The top 5 rules for X --> Y are:")
print(confidence_df[["freq_item_A", "freq_item_B", "confidence"]].head(5))

The top 5 rules for X --> Y are:
    freq_item_A freq_item_B  confidence
624    DAI93865    FRO40251    1.000000
134    GRO85051    FRO40251    0.999176
130    GRO38636    FRO40251    0.990654
591    ELE12951    FRO40251    0.990566
208    DAI88079    FRO40251    0.986726


In [ ]:
# select a list of candidates for frequent pairs (X,Y,Z) from frequent pairs (X,Y)
candidate_list = set(confidence_df.freq_item_A)

In [ ]:
# remove non-candidate items in each basket
basket_of_frequent_items_3 = items_no_dupes.map(lambda x: [i for i in x if i in candidate_list])
basket_of_frequent_items_3.take(3)

[['ELE17451', 'SNA90258', 'GRO99222'],
 ['ELE17451', 'SNA30755', 'ELE26917', 'GRO99222'],
 ['GRO73461', 'DAI22896', 'ELE17451', 'SNA99873']]

In [ ]:
# sort frequent items in each basket 
# (ordered items will avoid duplicates in (X,Y, Z) pairs)
basket_of_frequent_items_sorted_3 = basket_of_frequent_items_3.map(lambda x: sorted(x))
basket_of_frequent_items_sorted_3.take(3)

[['ELE17451', 'GRO99222', 'SNA90258'],
 ['ELE17451', 'ELE26917', 'GRO99222', 'SNA30755'],
 ['DAI22896', 'ELE17451', 'GRO73461', 'SNA99873']]

In [ ]:
# create pairs of frequent items (X,Y,Z) in each basket
# note that each basket is ordered
pair_freq_items_each_basket_3 = basket_of_frequent_items_sorted_3.map(lambda x: [(i,j, k) for i in x for j in x for k in x if i<j and j<k])
pair_freq_items_each_basket_3.take(3)

[[('ELE17451', 'GRO99222', 'SNA90258')],
 [('ELE17451', 'ELE26917', 'GRO99222'),
  ('ELE17451', 'ELE26917', 'SNA30755'),
  ('ELE17451', 'GRO99222', 'SNA30755'),
  ('ELE26917', 'GRO99222', 'SNA30755')],
 [('DAI22896', 'ELE17451', 'GRO73461'),
  ('DAI22896', 'ELE17451', 'SNA99873'),
  ('DAI22896', 'GRO73461', 'SNA99873'),
  ('ELE17451', 'GRO73461', 'SNA99873')]]

In [ ]:
# get a list of all pairs of frequent items (X,Y,Z)
list_pairs_freq_items_3 = pair_freq_items_each_basket_3.flatMap(lambda x:x).collect()
list_pairs_freq_items_3[:5]

[('ELE17451', 'GRO99222', 'SNA90258'),
 ('ELE17451', 'ELE26917', 'GRO99222'),
 ('ELE17451', 'ELE26917', 'SNA30755'),
 ('ELE17451', 'GRO99222', 'SNA30755'),
 ('ELE26917', 'GRO99222', 'SNA30755')]

In [ ]:
# count each (X,Y,Z) pair in the list
from collections import Counter
pair_of_frequent_items_count_3 = dict(Counter(list_pairs_freq_items_3))
pair_of_frequent_items_count_3

In [ ]:
# extract (X,Y,Z) pairs that have support >= 100
frequent_pairs_3 = [[key, val] for key, val in pair_of_frequent_items_count_3.items() if val >= 100]
frequent_pairs_3[:5]

[[('ELE17451', 'GRO30386', 'GRO73461'), 103],
 [('ELE17451', 'SNA59903', 'SNA72163'), 127],
 [('DAI62779', 'ELE17451', 'FRO78087'), 121],
 [('DAI62779', 'ELE17451', 'ELE26917'), 160],
 [('DAI62779', 'ELE17451', 'FRO31317'), 106]]

In [ ]:
# construct pd dataframe [table1] of frequent pairs (X,Y,Z) and counts
frequent_pairs_pd_3 = pd.DataFrame(frequent_pairs_3).rename(columns={0:"freq_pairs_XYZ", 1:"pair_count"})
nrow = frequent_pairs_pd_3.shape[0]
frequent_pairs_pd_3.head(3)

,freq_pairs_XYZ,pair_count
0,"(ELE17451, GRO30386, GRO73461)",103
1,"(ELE17451, SNA59903, SNA72163)",127
2,"(DAI62779, ELE17451, FRO78087)",121


In [ ]:
frequent_pairs_pd_3["freq_pairs_XY"] = [(frequent_pairs_pd_3.freq_pairs_XYZ[i][0], frequent_pairs_pd_3.freq_pairs_XYZ[i][1]) for i in range(nrow)]
frequent_pairs_pd_3["freq_pairs_XZ"] = [(frequent_pairs_pd_3.freq_pairs_XYZ[i][0], frequent_pairs_pd_3.freq_pairs_XYZ[i][2]) for i in range(nrow)]
frequent_pairs_pd_3["freq_pairs_YZ"] = [(frequent_pairs_pd_3.freq_pairs_XYZ[i][1], frequent_pairs_pd_3.freq_pairs_XYZ[i][2]) for i in range(nrow)]
frequent_pairs_pd_3.head(3)

,freq_pairs_XYZ,pair_count,freq_pairs_XY,freq_pairs_XZ,freq_pairs_YZ
0,"(ELE17451, GRO30386, GRO73461)",103,"(ELE17451, GRO30386)","(ELE17451, GRO73461)","(GRO30386, GRO73461)"
1,"(ELE17451, SNA59903, SNA72163)",127,"(ELE17451, SNA59903)","(ELE17451, SNA72163)","(SNA59903, SNA72163)"
2,"(DAI62779, ELE17451, FRO78087)",121,"(DAI62779, ELE17451)","(DAI62779, FRO78087)","(ELE17451, FRO78087)"


In [ ]:
# construct pd dataframe [table1] of frequent pairs (X,Y) and counts
frequent_pairs_pd = pd.DataFrame(frequent_pairs) \
                      .rename(columns={0:"pair_of_two", 1:"count"})
frequent_pairs_pd.head(3)

,pair_of_two,count
0,"(ELE17451, GRO99222)",148
1,"(ELE17451, SNA90258)",113
2,"(GRO99222, SNA90258)",156


In [ ]:
# join [table1] and [table2] to calculate conf(XY \rightarrow Z)
confidence_XYZ_df1 = pd.merge(frequent_pairs_pd_3, frequent_pairs_pd,
                              how="left", left_on="freq_pairs_XY", right_on="pair_of_two") \
                       .drop(columns=["freq_pairs_XY","freq_pairs_XZ", "freq_pairs_YZ"]) \
                       .rename(columns={"count":"count_XY",
                                        "pair_of_two":"freq_pairs_XY"})
confidence_XYZ_df1.head(3)

confidence_XYZ_df2 = pd.merge(frequent_pairs_pd_3, frequent_pairs_pd,
                              how="left", left_on="freq_pairs_XZ", right_on="pair_of_two") \
                       .drop(columns=["freq_pairs_XY","freq_pairs_XZ", "freq_pairs_YZ"]) \
                       .rename(columns={"count":"count_XY",
                                        "pair_of_two":"freq_pairs_XY"})
confidence_XYZ_df2.head(3) 

confidence_XYZ_df3 = pd.merge(frequent_pairs_pd_3, frequent_pairs_pd,
                              how="left", left_on="freq_pairs_YZ", right_on="pair_of_two") \
                       .drop(columns=["freq_pairs_XY","freq_pairs_XZ", "freq_pairs_YZ"]) \
                       .rename(columns={"count":"count_XY",
                                        "pair_of_two":"freq_pairs_XY"})
confidence_XYZ_df3.head(3) 

confidence_XYZ_df = confidence_XYZ_df1.append(confidence_XYZ_df2).append(confidence_XYZ_df3)

In [ ]:
# sanity check on num of rows
confidence_XYZ_df.shape[0] == confidence_XYZ_df1.shape[0]+confidence_XYZ_df2.shape[0]+confidence_XYZ_df3.shape[0]

True

In [ ]:
confidence_XYZ_df["confidence"] = confidence_XYZ_df.pair_count / confidence_XYZ_df.count_XY

In [ ]:
nrow = confidence_XYZ_df.shape[0]
confidence_XYZ_df["item_X"] = [confidence_XYZ_df.iloc[i].freq_pairs_XY[0] for i in range(nrow)]
confidence_XYZ_df["item_Y"] = [confidence_XYZ_df.iloc[i].freq_pairs_XY[1] for i in range(nrow)]

In [ ]:
confidence_XYZ_df["item_Z"] = [list(set(confidence_XYZ_df.iloc[i].freq_pairs_XYZ)-set(confidence_XYZ_df.iloc[i].freq_pairs_XY))[0] for i in range(nrow)]

In [ ]:
confidence_XYZ_df.sort_values(["confidence", "item_X","item_Y"], ascending = (False, True, True), inplace=True)

In [ ]:
print("The top 5 rules for (X,Y) --> Z are:")
print(confidence_XYZ_df[["freq_pairs_XYZ","freq_pairs_XY", "item_Z", "confidence" ]].head(5))

The top 5 rules for (X,Y) --> Z are:
                     freq_pairs_XYZ         freq_pairs_XY    item_Z  confidence
229  (DAI23334, DAI62779, ELE92920)  (DAI23334, ELE92920)  DAI62779         1.0
137  (DAI31081, FRO40251, GRO85051)  (DAI31081, GRO85051)  FRO40251         1.0
71   (DAI55911, FRO40251, GRO85051)  (DAI55911, GRO85051)  FRO40251         1.0
17   (DAI62779, DAI88079, FRO40251)  (DAI62779, DAI88079)  FRO40251         1.0
52   (DAI75645, FRO40251, GRO85051)  (DAI75645, GRO85051)  FRO40251         1.0
